In [ ]:
import json

raw_data_path = '/Users/shuogudaojin/Public/Database/SLAKE/chat_json/train.json'
with open(raw_data_path, 'r') as file:
    raw_data = json.load(file)

In [38]:
train_path = '/Users/shuogudaojin/Public/Database/SLAKE/chat_json/train_chat.json'
with open(train_path, 'r') as file:
    train = json.load(file)

In [94]:
import pandas as pd

raw_data_path = '/Users/shuogudaojin/Public/Database/SLAKE/chat_json/train.json'

raw_data_pandas = pd.read_json(raw_data_path)

In [98]:
# Get all of the topics
raw_data_pandas = pd.read_json(raw_data_path)
topics = raw_data_pandas['location']
topics = pd.unique(topics)
topics = topics.tolist()
print(topics)

['Abdomen', 'Lung', 'Chest_heart', 'Chest_lung', 'Brain_Tissue', 'Brain_Face', 'Brain', 'Neck', 'Chest_mediastinal', 'Pelvic Cavity']


In [100]:
raw_data_pandas.loc[raw_data_pandas['location'] == 'Brain_Tissue']

,img_id,img_name,question,answer,q_lang,location,modality,answer_type,base_type,content_type,triple,qid
2836,395,xmlab395/source.jpg,Is this image taken via MRI?,No,en,Brain_Tissue,CT,CLOSED,vqa,Modality,"[vhead, _, _]",2836
2837,395,xmlab395/source.jpg,"Which plane is the image scanned, transverse p...",Transverse Plane,en,Brain_Tissue,CT,CLOSED,vqa,Plane,"[vhead, _, _]",2837
2838,395,xmlab395/source.jpg,Which part of the body does this image belong to?,Head,en,Brain_Tissue,CT,OPEN,vqa,Position,"[vhead, _, _]",2838
2839,395,xmlab395/source.jpg,Where are the eyes in this image?,Top,en,Brain_Tissue,CT,OPEN,vqa,Position,"[vhead, _, _]",2839
2840,395,xmlab395/source.jpg,Which organ system is imaged?,Head,en,Brain_Tissue,CT,OPEN,vqa,Organ,"[vhead, _, _]",2840
...,...,...,...,...,...,...,...,...,...,...,...,...
11744,535,xmlab535/source.jpg,这张图像是关于颈部吗?,不是,zh,Brain_Tissue,MRI,CLOSED,vqa,Organ,"[vhead, _, _]",11744
11745,535,xmlab535/source.jpg,图中脑部左侧显示什么疾病?,脑水肿,zh,Brain_Tissue,MRI,OPEN,vqa,Abnormality,"[vhead, _, _]",11745
11746,535,xmlab535/source.jpg,"相对于其他组织,脑水肿是白色的还是灰色的?",白色,zh,Brain_Tissue,MRI,CLOSED,vqa,Color,"[vhead, _, _]",11746
11747,535,xmlab535/source.jpg,该图像中有多少种异常?,1,zh,Brain_Tissue,MRI,OPEN,vqa,Quantity,"[vhead, _, _]",11747


In [64]:
topics = raw_data_pandas['location']
topics = pd.unique(topics)
topics = topics.tolist()

In [65]:
topics

['Abdomen',
 'Lung',
 'Chest_heart',
 'Chest_lung',
 'Brain_Tissue',
 'Brain_Face',
 'Brain',
 'Neck',
 'Chest_mediastinal',
 'Pelvic Cavity']

In [90]:
import os 
import json
import random
import numpy as np

def non_iid_split_dirichlet(raw_data, train, topics, ideal_counts, n_clients, alpha=0.4):
    '''iid split if alpha is large; non-iid split if alpha is small'''
    
    # Split raw data idx into n_clients 
    client = []
    clients = {}
    total_samples = len(raw_data)
    idx_train = [value['qid'] for value in train]
    if 'Chest_mediastinal' in topics:
        topics.remove('Chest_mediastinal')
    
    if ideal_counts < 1:
        err = f'[SIMULATE] Decrease the number of participating clients (`n_clients` < {n_clients})!'
        raise Exception(err)
    
    #=============== Split the idx of the raw data by topics,and drop marginal topics =============#
    idx_raw_data_split = {}
    for value in topics:
        idx_raw_data_split[value] = []

    for value in raw_data:
        if value['location'] != 'Chest_mediastinal':
            idx_raw_data_split[value['location']].append(value)
        
    # Sort the topics of raw_data by length of its value
    idx_raw_data_split = dict(sorted(idx_raw_data_split.items(),
                                    key=lambda item: len(item[1])))

    # Delete the value with length less than 100
    # idx_raw_data_split = {key: value for key, value in idx_raw_data_split.items() if len(value) > 100}
    for key in list(idx_raw_data_split.keys()):
        if len(idx_raw_data_split[key]) < 100:
            del idx_raw_data_split[key]
            if key in topics:
                topics.remove(key)
            
    n_classes = len(topics) # number of different classes 
    #================ Split the raw_data into n_clients ===============#
    for k in range(n_clients):
        ### define Dirichlet distribution of which prior distribution is an uniform distribution
        diri_prior = np.random.uniform(size=n_classes)
        
        ### sample a parameter corresponding to that of categorical distribution
        cat_param = np.random.dirichlet(alpha * diri_prior)

        ### try to sample by amount of `ideal_counts`
        sampled = np.random.choice(n_classes, ideal_counts, p=cat_param)

        ### count per-class samples
        unique, counts = np.unique(sampled, return_counts=True)
        # print(unique)
        # print(counts)
        
        for idx, selected_class in enumerate(unique):
            selected_class = int(selected_class)
            selected_topic = topics[idx]
            
            if counts[idx] == 0: continue
            ### avoid the samples selected are more than the original samples
            samples_size_of_this_class = len(idx_raw_data_split[topics[idx]])
            if counts[idx] > samples_size_of_this_class:
                counts[idx] = samples_size_of_this_class
            
            # Get the samples and drop samples don't belong to idx_train
            selected_samples = random.sample(idx_raw_data_split[selected_topic], counts[idx])
            # print(selected_samples)
            # Element from selected_class is str, element from idx_train is str
            idx_train = [int(idx) for idx in idx_train]
            selected_samples = [int(item['qid']) for item in selected_samples]
            for idx in selected_samples:
                if idx not in idx_train:
                    selected_samples.remove(idx)
            # selected_samples = [idx for idx in selected_class if idx in idx_train]
            
            for idx in selected_samples:  
                for sample in train:
                    if sample['qid'] == str(idx):   
                        client.append(sample)
                        break
                
        # get client k
        print(len(client))
        clients[str(k)] = client
        client = []
    
    return clients

In [91]:
clients = non_iid_split_dirichlet(raw_data=raw_data, train=train, topics=topics,
                                  ideal_counts=1000, n_clients=5, alpha=100)

TypeError: 'int' object is not subscriptable

In [85]:
saving_folder = '/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA'
def save_clients_json(saving_folder, clients):
    os.makedirs(saving_folder, exist_ok=True)
    
    for key, value in clients.items():
        client_name = 'client_' + key + '.json'
        saving_path = os.path.join(saving_folder, client_name)
        print(f'The saving paths are:\n{saving_path}')
        with open(saving_path, 'w') as file:
            json.dump(value, file)
save_clients_json(saving_folder, clients)

The saving paths are:
/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA/client_0.json
The saving paths are:
/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA/client_1.json
The saving paths are:
/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA/client_2.json
The saving paths are:
/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA/client_3.json
The saving paths are:
/Users/shuogudaojin/Public/Database/SLAKE/iid_clients_with_only_VQA/client_4.json
